# CDA Python: Features & Examples
---

In [1]:
from ipywidgets import Dropdown
from IPython.display import display
tester_check = Dropdown(
    options=[True,False],
    description='Tester:',
    value=True,
)
display(tester_check)

Dropdown(description='Tester:', options=(True, False), value=True)

The following examples illustrate some ```CDA Python``` features while providing insights into the underlying data structure (**Getting started**). To demonstrate those features, we provide a few relevant text queries along with step-by-step explanations on how to translate those into the ```CDA Python``` queries (**Example queries**). Finally, there are a few additional queries intended for the test users to play around with and send feedback to the CDA team (**Test queries**).

In [2]:
from cdapython import Q, columns, unique_terms,query
import cdapython
print(cdapython.__file__)
print(cdapython.__version__)

/Users/elijahlowe/opt/miniconda3/envs/cda/lib/python3.9/site-packages/cdapython/__init__.py
2022.2.23


In [3]:

if(tester_check.value):
    Q.set_host_url("http://35.192.60.10:8080/")
else:
    Q.set_host_url("https://cda.cda-dev.broadinstitute.org")

print(Q.get_host_url())

http://35.192.60.10:8080/


# Counts

We can start by getting the record for ```id = TCGA-E2-A10A``` that we mentioned earlier:

In [4]:
q = Q('id = "TCGA-E2-A10A"') # note the double quotes for the string value

r = q.run()


q.counts()


Getting results from database

Total execution time: 3452 ms



        QueryID: 1b03e942-3e1f-417d-b25c-72f08e0276d6
        Query: SELECT
  top_level_file.system,
  CASE
    WHEN subject_count IS NULL THEN 0
    ELSE subject_count.count_file
  END AS subject_count,
  CASE
    WHEN top_level_file.count_file IS NULL THEN 0
    ELSE top_level_file.count_file
  END AS subject_file_count,
  CASE
    WHEN researchsubject_count.count_researchsubject IS NULL THEN 0
    ELSE researchsubject_count.count_researchsubject
  END AS researchsubject_count,
  CASE
    WHEN researchsubject_file_count.count_researchsubject IS NULL THEN 0
    ELSE researchsubject_file_count.count_researchsubject
  END AS researchsubject_file_count,
  CASE
    WHEN specimen_count.count_specimen IS NULL THEN 0
    ELSE specimen_count.count_specimen
  END AS specimen_count,
  CASE
    WHEN specimen_file_count.count_file IS NULL THEN 0
    ELSE specimen_file_count.count_file
  END AS specimen_file_count
FROM
  (
    SELECT
      _file_identifier.system,
      COUNT(_file_identifier.val

- The text needs correcting, `specimen_count Count`. 
- PDC has duplicates at the specimen level as indicated by `specimen_count Count: 2`
- SQL output should have a option to be hidden

### Zero counts breaks everything

In [5]:
q1 = Q('ResearchSubject.Specimen.primary_disease_type = "Nevi and Melanomas"')

q = q1
r = q.counts()

print(r)

(500)
Reason: 
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Date': 'Wed, 23 Feb 2022 16:30:00 GMT', 'Connection': 'close'})
HTTP response body: {"message":"Unrecognized name: _Specimen at [37:8]","statusCode":500,"causes":["Unrecognized name: _Specimen at [37:8]","400 Bad Request\nGET https://www.googleapis.com/bigquery/v2/projects/gdc-bq-sample/queries/7a40fd48-0f33-464f-bf7a-db0ffadd92f6?location=US&maxResults=0&prettyPrint=false\n{\n  \"code\" : 400,\n  \"errors\" : [ {\n    \"domain\" : \"global\",\n    \"location\" : \"q\",\n    \"locationType\" : \"parameter\",\n    \"message\" : \"Unrecognized name: _Specimen at [37:8]\",\n    \"reason\" : \"invalidQuery\"\n  } ],\n  \"message\" : \"Unrecognized name: _Specimen at [37:8]\",\n  \"status\" : \"INVALID_ARGUMENT\"\n}"]}

None


# Case sensitivity

In [6]:
q = Q('id = "tcga-E2-A10A"') # note the double quotes for the string value

r = q.run()

q.run()

Getting results from database

Total execution time: 3379 ms
Getting results from database

Total execution time: 3298 ms



        QueryID: 17696c7c-a5b8-46bc-93af-0a789a9f2ec3
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1 WHERE (UPPER(all_v2_1.id) = UPPER('tcga-E2-A10A'))
        Offset: 0
        Count: 1
        Total Row Count: 1
        GDC Count: 59 
 	PDC Count: 151 
 	IDC Count: 0
        More pages: False
        

In [7]:
StageIV = Q('ResearchSubject.Diagnosis.tumor_stage = "Stage IV"')
r = StageIV.run()
print(r)

 This Value ResearchSubject.Diagnosis.tumor_stage has been deprecated but will be converted it for you in the background please use the new value ResearchSubject.Diagnosis.stage
Getting results from database

Total execution time: 10424 ms

        QueryID: 6c17be1c-2cc4-4e3f-8c87-3476c682e754
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE (_Diagnosis.stage = UPPER('Stage IV'))
        Offset: 0
        Count: 0
        Total Row Count: 0
        No counts could be found
        More pages: False
        


In [8]:
stage = Q('ResearchSubject.Diagnosis.tumor_stage = "Stage IIIC"')
r = stage.run(host = 'https://cda.cda-dev.broadinstitute.org')
print(r)

 This Value ResearchSubject.Diagnosis.tumor_stage has been deprecated but will be converted it for you in the background please use the new value ResearchSubject.Diagnosis.stage
Getting results from database

Total execution time: 28175 ms

        QueryID: 4f138bfa-abb1-4884-8028-55625b8d3307
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE (_Diagnosis.stage = 'Stage IIIC')
        Offset: 0
        Count: 92
        Total Row Count: 92
        GDC Count: 3112 
 	PDC Count: 12434 
 	IDC Count: 3243
        More pages: False
        


In [14]:
stage = Q('ResearchSubject.Diagnosis.stage = "Stage IIIC"')
r = stage.run()
print(r)

Getting results from database

Total execution time: 3187 ms

        QueryID: 5afa2759-7a9d-40d7-99d6-185fadd936d1
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE (_Diagnosis.stage = UPPER('Stage IIIC'))
        Offset: 0
        Count: 0
        Total Row Count: 0
        No counts could be found
        More pages: False
        


case senitivity seems to break at stages

# Backwards compatibility

In [7]:
unique_terms('ResearchSubject.Diagnosis.tumor_stage')

 This Value ResearchSubject.Diagnosis.tumor_stage has been deprecated but will be converted it for you in the background please use the new value ResearchSubject.Diagnosis.stage


[None,
 'IA',
 'IB',
 'IIA',
 'IIB',
 'IIIA',
 'IIIB',
 'IV',
 'IVB',
 'N/A',
 'Not Performed',
 'Not Received',
 'Not Reported',
 'Not Reported/ Unknown',
 'Not Reported/Unknown',
 'Stage 1B',
 'Stage I',
 'Stage IA',
 'Stage IA3',
 'Stage IB',
 'Stage IC',
 'Stage II',
 'Stage IIA',
 'Stage IIB',
 'Stage III',
 'Stage IIIA',
 'Stage IIIB',
 'Stage IIIC',
 'Stage IV',
 'Stage IVA',
 'Stage1',
 'Unknown',
 'no resection',
 'pT1',
 'pT1a',
 'pT1b',
 'pT2 N0',
 'pT2, pN2, pM not applicable',
 'pT2N1',
 'pT2N2',
 'pT3 N2 M(Not Applicable)',
 'pT3a',
 'pT4']

Backwards compatibility doesn't work for `unique_terms()`

In [11]:
unique_terms('ResearchSubject.Diagnosis.stage')

[None,
 'IA',
 'IB',
 'IIA',
 'IIB',
 'IIIA',
 'IIIB',
 'IV',
 'IVB',
 'N/A',
 'Not Performed',
 'Not Received',
 'Not Reported',
 'Not Reported/ Unknown',
 'Not Reported/Unknown',
 'Stage 1B',
 'Stage I',
 'Stage IA',
 'Stage IA3',
 'Stage IB',
 'Stage IC',
 'Stage II',
 'Stage IIA',
 'Stage IIB',
 'Stage III',
 'Stage IIIA',
 'Stage IIIB',
 'Stage IIIC',
 'Stage IV',
 'Stage IVA',
 'Stage1',
 'Unknown',
 'no resection',
 'pT1',
 'pT1a',
 'pT1b',
 'pT2 N0',
 'pT2, pN2, pM not applicable',
 'pT2N1',
 'pT2N2',
 'pT3 N2 M(Not Applicable)',
 'pT3a',
 'pT4']

In [8]:
q1 = Q('ResearchSubject.Diagnosis.age_at_diagnosis > 50*365')
q2 = Q('ResearchSubject.associated_project = "TCGA-OV"')
q = q1.And(q2)
r = q.run()
print(r)

 This Value ResearchSubject.associated_project has been deprecated but will be converted it for you in the background please use the new value ResearchSubject.member_of_research_project
Getting results from database

Total execution time: 56063 ms

        QueryID: 1e5601e9-5e65-416c-bf9f-9be5dfc3ae40
        Query: SELECT all_v2_1.* FROM gdc-bq-sample.integration.all_v2_1 AS all_v2_1, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_Diagnosis.age_at_diagnosis > 50*365) AND (_ResearchSubject.member_of_research_project = UPPER('TCGA-OV')))
        Offset: 0
        Count: 100
        Total Row Count: 461
        GDC Count: 5197 
 	PDC Count: 4833 
 	IDC Count: 10505
        More pages: True
        
